In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

#np.set_printoptions(precision=4)
import math
from utils import *

nS = 8
nA = 2
seed = 3231
gamma = .9
river = River(nS, gamma=gamma, small=5, large=1000, seed=seed)
tau = 0.9
xi = np.ones(nS) * 1/nS
tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)


In [2]:
print(tmdp.s)
s_prime, r, flags, prob = tmdp.step(1)
print(s_prime, r, flags, prob)

[1]
[1] [0.] {'done': False, 'teleport': False} [0.1725]


In [3]:
r_s_a = compute_r_s_a(tmdp.P_mat_tau, tmdp.reward)
Q_star, _ = bellman_optimal_q(tmdp.P_mat_tau, tmdp.reward, gamma).values()
pi = get_policy(Q_star)
pi_prime = np.array([[0., 1.] for i in range(nS)])

V_star = get_value_function(Q_star)
print(tmdp.P_mat.shape)
print(Q_star)


(8, 2, 8)
[[155.39191625 154.30072883]
 [154.32941634 154.24077201]
 [154.23379139 154.22581344]
 [154.2251852  154.22455752]
 [154.22441068 154.22775533]
 [154.22464204 154.34392193]
 [154.23509708 158.41319717]
 [154.60133189 300.94982072]]


In [4]:
Q = np.zeros((nS, nA))
Qs = Q_learning_2(tmdp, Q_star, alpha=1, episodes=10000000, status_step=100000)



In [5]:
Q =Qs["Qs"][-1]

In [6]:
d = compute_d(tmdp.mu, tmdp.P_mat_tau, pi, tmdp.gamma)
delta = compute_delta(d, pi)
print(delta)
tmdp.s.item()
#tmdp.allowed_actions[tmdp.s]

[[0.1373545  0.        ]
 [0.12491773 0.        ]
 [0.12408586 0.        ]
 [0.11484289 0.        ]
 [0.         0.121432  ]
 [0.         0.12496323]
 [0.         0.1318389 ]
 [0.         0.1205649 ]]


2

In [12]:
U = get_state_action_nextstate_value_function(tmdp.P_mat_tau, tmdp.reward, tmdp.gamma, Q_star)
q_reb = rebuild_Q_from_U(tmdp.P_mat_tau, U)
print(q_reb)
print(Q)
print(Q_star- Q)

[[153.60806592 152.48400703]
 [152.54556592 152.52670423]
 [152.34036961 152.7139402 ]
 [152.89308622 152.58088973]
 [152.48017169 152.69812426]
 [152.67826032 152.90945123]
 [152.810503   156.02878411]
 [153.18441132 298.56617494]]
[[155.2974098  135.0753007 ]
 [132.95172069 153.0174508 ]
 [159.15874647 131.76943798]
 [154.57080718 131.51604721]
 [134.01174428 156.77179204]
 [134.96410872 158.24115514]
 [134.88635184 162.39569204]
 [151.45820269 269.1276778 ]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [8]:
model_adv = get_model_advantage_function(tmdp.P_mat_tau, tmdp.reward, tmdp.gamma, Q_star)
rel_model_adv_hat = compute_relative_model_advantage_function_hat(tmdp.P_mat, tmdp.xi, U_star)
# With the fixed policy, evaluate the relative model advantage function in moving from P_mat_tau to P_mat, that is, moving from the simplified problem
# with tau != 0 to the original problem with tau = 0
rel_model_adv = compute_relative_model_advantage_function(tmdp.P_mat, model_adv)


print(compute_discounted_distribution_relative_model_advantage_function_from_delta_tau(rel_model_adv_hat, delta, tmdp.tau, 0.))
print(compute_discounted_distribution_relative_model_advantage_function(rel_model_adv, delta))

NameError: name 'U_star' is not defined

In [ ]:
dif = get_expected_difference_transition_models(tmdp.P_mat_tau, tmdp.P_mat, delta)
print(dif)
print(tmdp.P_mat_tau.shape)

expected_value = np.sum(delta.T * np.abs(tmdp.P_mat - tmdp.P_mat_tau))

print(expected_value)
dif_tau = get_expected_difference_transition_models(tmdp.P_mat, tmdp.xi, delta)
print(dif_tau*tmdp.tau)


1.394688077659496
(8, 2, 8)
1.5092592543131351
1.394688077659496


In [ ]:
sup = get_sup_difference_transition_models(tmdp.P_mat_tau, tmdp.P_mat)
print(sup)

print(np.max(np.abs(tmdp.P_mat_tau - tmdp.P_mat)))

Xi = np.tile(tmdp.xi, (nA, nS)).T
Xi = Xi.reshape((nS, nA, nS))

print(np.max(np.abs(tmdp.P_mat - Xi))*tmdp.tau)

0.7875000000000001
0.7875000000000001
0.7875
